In [1]:
import os
import sys
import pandas as pd

package_path = '/Users/flaminia/Documents/work/ProMCDA'

if package_path not in sys.path:
    sys.path.append(package_path)

try:
    from mcda.models.ProMCDA import ProMCDA
    print("Import successful!")
except ModuleNotFoundError as e:
    print(f"ModuleNotFoundError: {e}")

from mcda.configuration.enums import NormalizationFunctions, AggregationFunctions, OutputColumnNames4Sensitivity, NormalizationNames4Sensitivity, PDFType

Import successful!


In [2]:
def setUp():
    
    # Mock input data for testing
    input_matrix_without_uncertainty = pd.DataFrame({
        'Criteria 1': [0.5, 0.2, 0.8],
        'Criteria 2': [0.3, 0.6, 0.1]
    }, index=['A', 'B', 'C'])

    input_matrix_with_uncertainty = pd.DataFrame({
    'alternatives': ['alt1', 'alt2', 'alt3', 'alt4'],
    'ind1_min': [-15.20, -12.40, 10.60, -39.70],
    'ind1_max': [8.20, 8.70, 2.00, 14.00],
    'ind2': [0.04, 0.05, 0.11, 0.01],
    'ind3_average': [24.50, 24.50, 14.00, 26.50],
    'ind3_std': [6.20, 4.80, 0.60, 4.41],
    'ind4_average': [-15.20, -12.40, 1.60, -39.70],
    'ind4_std': [8.20, 8.70, 2.00, 14.00],
    'ind5': [0.04, 0.05, 0.11, 0.01],
    'ind6_average': [24.50, 24.50, 14.00, 26.50],
    'ind6_std': [6.20, 4.80, 0.60, 4.41]
    })
   
    input_matrix_with_uncertainty.set_index('alternatives', inplace=True)

    polarity = ('+', '-')

    robustness_weights = False
    robustness_indicators = False

    marginal_distributions = [PDFType.NORMAL, PDFType.NORMAL]

    num_runs = 5
    num_cores = 1

    output_path = 'mock_output/'

    # Return the setup parameters as a dictionary
    return {
        'input_matrix': input_matrix_without_uncertainty, # Decide what type of input matrix
        'polarity': polarity,
        'robustness_weights': robustness_weights,
        'robustness_indicators': robustness_indicators,
        'marginal_distributions': marginal_distributions,
        'num_runs': num_runs,
        'num_cores': num_cores,
        'output_path': output_path
    }

# Run the setup and store parameters in a variable
setup_parameters = setUp()

# Check the setup parameters
setup_parameters

{'input_matrix':    Criteria 1  Criteria 2
 A         0.5         0.3
 B         0.2         0.6
 C         0.8         0.1,
 'polarity': ('+', '-'),
 'robustness_weights': False,
 'robustness_indicators': False,
 'marginal_distributions': [<PDFType.NORMAL: 'normal'>,
  <PDFType.NORMAL: 'normal'>],
 'num_runs': 5,
 'num_cores': 1,
 'output_path': 'mock_output/'}

In [3]:
promcda = ProMCDA(
    input_matrix=setup_parameters['input_matrix'],
    polarity=setup_parameters['polarity'],
    robustness_weights=setup_parameters['robustness_weights'],
    robustness_indicators=setup_parameters['robustness_indicators'],
    marginal_distributions=setup_parameters['marginal_distributions'],
    num_runs=setup_parameters['num_runs'],
    num_cores=setup_parameters['num_cores'],
    #output_path=setup_parameters['output_path']
)

INFO: 2024-12-03 17:26:36,915 - ProMCDA - Alternatives are ['A', 'B', 'C']


In [4]:
#promcda.normalize()

In [5]:
promcda.normalize(NormalizationFunctions.MINMAX)

,Criteria 1_minmax_01,Criteria 2_minmax_01,Criteria 1_minmax_without_zero,Criteria 2_minmax_without_zero
0,0.5,0.6,0.55,0.64
1,0.0,0.0,0.1,0.1
2,1.0,1.0,1.0,1.0


In [6]:
print(promcda.normalized_values_without_robustness)

  Criteria 1_minmax_01 Criteria 2_minmax_01 Criteria 1_minmax_without_zero  \
0                  0.5                  0.6                           0.55   
1                  0.0                  0.0                            0.1   
2                  1.0                  1.0                            1.0   

  Criteria 2_minmax_without_zero  
0                           0.64  
1                            0.1  
2                            1.0  


In [7]:
promcda.aggregate(aggregation_method=AggregationFunctions.WEIGHTED_SUM, weights=None)

Normalization method minmax
Entering weighted sum
Columns:   Criteria 1_minmax_01 Criteria 2_minmax_01
0                  0.5                  0.6
1                  0.0                  0.0
2                  1.0                  1.0
Normalized indicators shape: (3, 2)
Weights: [0.5, 0.5]


,minmax_weighted_sum
0,0.55
1,0.0
2,1.0
